In [ ]:
import os

import numpy as np
import pandas as pd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
os.chdir('/home/adam/adam/causal_inference')

from causal_inference.make_data.make_data import UseCaseLoader

from causal_inference.make_data.make_artificial_sessions import *
from causal_inference.make_data.make_proning_sessions import *
from causal_inference.make_data.make_covariates import *

In [ ]:
dl = UseCaseLoader()

In [ ]:
raw_data_path = '/home/adam/adam/data/causal_inference/data/raw/position_measurements.csv'
data_sessions_path = '/home/adam/adam/data/causal_inference/data/interim/unique_sessions_test.csv'
data_artificial_sessions = '/home/adam/adam/data/causal_inference/data/interim/artificial_sessions_test.csv'
data_inclusion_path = '/home/adam/adam/data/causal_inference/data/interim/incusion_sessions_test.csv'
data_covariates_path = '/home/adam/adam/data/causal_inference/data/interim/covariates.csv'
data_outcome_path = '/home/adam/adam/data/causal_inference/data/interim/outcomes.csv'

In [ ]:
dl.get_position_measurements(save_path=raw_data_path)

In [ ]:
dl.make_unique_sessions(load_path=raw_data_path,
                        save_path=data_sessions_path)

In [ ]:
dl.add_artificial_sessions(load_path=data_sessions_path,
                           save_path=data_artificial_sessions)

In [ ]:
dl.add_inclusion_criteria(load_path=data_artificial_sessions,
                          save_path=data_inclusion_path)

In [ ]:
dl.add_covariates(load_path=data_inclusion_path, save_path=data_covariates_path)


In [ ]:
dl.add_outcomes(load_path=data_covariates_path, save_path=data_outcome_path)

In [ ]:
df = pd.read_csv(data_sessions_path)

In [ ]:
df.head()

In [ ]:
for row in df.itertuples():
    print(row.Index, type(row.start_timestamp))

In [ ]:
for row in df.itertuples(index=False):
    print(type(row.start_timestamp))


In [ ]:
from causal_inference.make_data.make_proning_sessions import load_raw_position_data

load_raw_position_data(raw_data_path)

In [ ]:
df['episode_id'].unique().tolist()